<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/2025_06_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -o "/content/drive/MyDrive/open_1.zip" -d "/content/open_1"


Archive:  /content/drive/MyDrive/open_1.zip
  inflating: /content/open_1/sample_submission.csv  
  inflating: /content/open_1/test.csv  
  inflating: /content/open_1/train.csv  


In [3]:
import os

def get_path(filename):
    return "/content/open_1/" + filename


In [4]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm optuna
!pip install -q lightgbm catboost xgboost



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.5 MB/s eta 0:00:00


In [5]:
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [6]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score, precision_recall_curve
from sklearn.calibration import CalibratedClassifierCV
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 (라벨 인코딩 + 결측치 평균 대체)
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ 클래스 비율에 따른 가중치 계산
pos_weight = (y == 0).sum() / (y == 1).sum()

# ✅ base 모델 정의 (불균형 대응 포함)
model_lgbm = LGBMClassifier(class_weight="balanced", learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(scale_pos_weight=pos_weight, use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(class_weights=[1, pos_weight], verbose=0, random_state=42)

# ✅ meta 모델
meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

# ✅ Stacking 모델 구성
base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ OOF 예측으로 PR-curve 기반 최적 threshold 탐색
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
precision, recall, thresholds = precision_recall_curve(y, oof_preds)
f1_scores = 2 * precision * recall / (precision + recall + 1e-6)
best_thresh = thresholds[np.argmax(f1_scores)]
print(f"✅ Best Threshold: {best_thresh:.4f} | F1(O.O.F.): {max(f1_scores):.5f}")

# ✅ 스태킹 전체 학습
stack_model.fit(X, y)

# ✅ 확률 보정 (Isotonic 방법 추천)
calibrated = CalibratedClassifierCV(estimator=stack_model, method='isotonic', cv=5)
calibrated.fit(X, y)

# ✅ 보정된 확률로 테스트셋 예측
test_preds = calibrated.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)

# ✅ 저장
submission.to_csv("submission_calibrated_stacking.csv", index=False)
print("✅ 'submission_calibrated_stacking.csv' 저장 완료")


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003110 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8368, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001987 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69728, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120009 -> initscore=-1.992343
[LightGBM] [Info] Start training from score -1.992343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ 'submission_calibrated_stacking.csv' 저장 완료


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [9]:
from google.colab import files
files.download("submission_calibrated_stacking.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder, RobustScaler, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 설정
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리
def preprocess(df, encoders=None, is_train=True):
    df = df.copy()
    encoders = encoders or {}
    num_cols = df.select_dtypes(include=["int64", "float64"]).columns
    cat_cols = df.select_dtypes(include=["object"]).columns

    for col in num_cols:
        df[col] = SimpleImputer(strategy="median").fit_transform(df[[col]])

    for col in cat_cols:
        df[col] = df[col].astype(str)
        if is_train:
            encoders[col] = LabelEncoder().fit(df[col])
        df[col] = encoders[col].transform(df[col])

    df[num_cols] = RobustScaler().fit_transform(df[num_cols])
    return df, encoders

X, enc = preprocess(X)
X_test, _ = preprocess(X_test, enc, is_train=False)

# ✅ Base 모델 정의
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)

# ✅ 메타 모델: 확률 지원 가능한 모델 사용
meta_model = LGBMClassifier(learning_rate=0.01, max_depth=3, num_leaves=15, random_state=42)

# ✅ Stacking 모델 구성
stack_model = StackingClassifier(
    estimators=[
        ('lgbm', model_lgbm),
        ('xgb', model_xgb),
        ('cat', model_cat)
    ],
    final_estimator=meta_model,
    passthrough=True,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ OOF 예측 + threshold 튜닝
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"✅ Best Threshold: {best_thresh:.3f}, F1 Score: {max(f1s):.5f}")

# ✅ 전체 학습 및 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]

# ✅ 확률 보정 (QuantileTransformer)
qt = QuantileTransformer(output_distribution='uniform', random_state=42)
test_preds_adj = qt.fit_transform(test_preds.reshape(-1, 1)).flatten()

# ✅ 결과 저장
submission["Cancer"] = (test_preds_adj > best_thresh).astype(int)
submission.to_csv("submission_final_stack_calibrated.csv", index=False)
print("📦 저장 완료: submission_final_stack_calibrated.csv")


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008920 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1888
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from google.colab import files
files.download("submission_final_stack_calibrated.csv")
